<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/145_B2B_Sales_Agent_Claude_Langchain_03_Personalization_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
LangChain Personalization Agent - Creates custom outreach messages using templates and chains

This agent demonstrates:
- LangChain prompt templates
- Chain composition for message generation
- Structured output with Pydantic models
- Template management and customization
"""

import logging
from typing import List, Dict, Any, Optional
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel as PydanticV1BaseModel
from langchain_models import (
    CompanyInfo, AnalysisResult, PersonalizationResult, OutreachMessage,
    PersonalizationStrategy, MessageChannel, MessageTone
)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PersonalizationInput(PydanticV1BaseModel):
    """Input schema for personalization"""
    company_info: str
    analysis_result: str
    sender_name: str

class LangChainPersonalizationAgent:
    """
    LangChain Personalization Agent that uses templates and chains to create outreach messages

    This demonstrates:
    - LangChain prompt templates for message generation
    - Chain composition for complex personalization workflows
    - Template management and customization
    - Structured output with Pydantic models
    """

    def __init__(self, agent_id: str = "langchain_personalization_agent", use_mock: bool = True):
        self.agent_id = agent_id
        self.logger = logging.getLogger(f"{__name__}.{agent_id}")
        self.use_mock = use_mock

        if use_mock:
            # Use mock LLM for demonstration
            self.llm = None
            self.logger.info("Using mock LLM for demonstration")
        else:
            # Initialize OpenAI LLM
            self.llm = ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.3,
                max_tokens=500
            )
            self.logger.info("Using OpenAI LLM")

        # Initialize prompt templates
        self._setup_prompt_templates()

        # Initialize chains
        self._setup_chains()

        self.logger.info(f"LangChain Personalization Agent initialized: {agent_id}")

    def _setup_prompt_templates(self):
        """Set up prompt templates for personalization"""

        # Email template for pain point focused approach
        self.email_pain_point_template = PromptTemplate(
            input_variables=["contact_name", "company_name", "industry", "pain_point", "solution", "sender_name"],
            template="""
Write a professional email to {contact_name} at {company_name} about {pain_point}.

Company: {company_name}
Industry: {industry}
Pain Point: {pain_point}
Solution: {solution}
Sender: {sender_name}

Email should be:
- Professional but not overly formal
- Focused on the specific pain point
- Include a clear call to action
- Be personalized to the contact and company
- Keep it concise (under 150 words)

Format:
Subject: [compelling subject line]
Body: [email body]
"""
        )

        # Email template for opportunity focused approach
        self.email_opportunity_template = PromptTemplate(
            input_variables=["contact_name", "company_name", "industry", "opportunity", "value", "sender_name"],
            template="""
Write a professional email to {contact_name} at {company_name} about {opportunity}.

Company: {company_name}
Industry: {industry}
Opportunity: {opportunity}
Value: {value}
Sender: {sender_name}

Email should be:
- Professional and consultative
- Focused on the opportunity
- Include a clear call to action
- Be personalized to the contact and company
- Keep it concise (under 150 words)

Format:
Subject: [compelling subject line]
Body: [email body]
"""
        )

        # LinkedIn connection request template
        self.linkedin_template = PromptTemplate(
            input_variables=["contact_name", "company_name", "industry", "sender_name"],
            template="""
Write a LinkedIn connection request message to {contact_name} at {company_name}.

Company: {company_name}
Industry: {industry}
Contact: {contact_name}
Sender: {sender_name}

Message should be:
- Professional and brief
- Mention the company or industry
- Include a reason for connecting
- Be under 100 characters

Format:
[connection request message]
"""
        )

        # Strategy selection template
        self.strategy_template = PromptTemplate(
            input_variables=["pain_points", "opportunities", "company_size"],
            template="""
Based on the following information, recommend the best personalization strategy:

Pain Points: {pain_points}
Opportunities: {opportunities}
Company Size: {company_size}

Choose one strategy:
1. pain_point_focused - If there are high-severity pain points
2. opportunity_focused - If there are high-priority opportunities
3. relationship_building - If neither pain points nor opportunities are compelling

Return only the strategy name.
"""
        )

    def _setup_chains(self):
        """Set up LangChain chains"""
        if not self.use_mock:
            self.email_pain_point_chain = LLMChain(
                llm=self.llm,
                prompt=self.email_pain_point_template,
                output_key="email_message"
            )

            self.email_opportunity_chain = LLMChain(
                llm=self.llm,
                prompt=self.email_opportunity_template,
                output_key="email_message"
            )

            self.linkedin_chain = LLMChain(
                llm=self.llm,
                prompt=self.linkedin_template,
                output_key="linkedin_message"
            )

            self.strategy_chain = LLMChain(
                llm=self.llm,
                prompt=self.strategy_template,
                output_key="strategy"
            )

    def personalize_outreach(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                           sender_name: str = "Sales Professional") -> PersonalizationResult:
        """
        Create personalized outreach messages using LangChain templates and chains

        Args:
            company_info: CompanyInfo from Research Agent
            analysis_result: AnalysisResult from Analysis Agent
            sender_name: Name of the person sending outreach

        Returns:
            PersonalizationResult with customized messages
        """
        try:
            self.logger.info(f"Creating LangChain personalized outreach for {company_info.name}")

            if self.use_mock:
                # Use mock personalization for demonstration
                return self._mock_personalization(company_info, analysis_result, sender_name)
            else:
                # Use real LLM chains
                return self._llm_personalization(company_info, analysis_result, sender_name)

        except Exception as e:
            self.logger.error(f"LangChain personalization failed for {company_info.name}: {str(e)}")
            raise

    def _mock_personalization(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                            sender_name: str) -> PersonalizationResult:
        """Mock personalization for demonstration purposes"""

        # Select primary contact
        primary_contact = self._select_primary_contact(company_info, analysis_result)

        if not primary_contact:
            self.logger.warning(f"No contact information available for {company_info.name}")
            return PersonalizationResult(
                company_name=company_info.name,
                primary_contact=None,
                messages=[],
                personalization_strategy=PersonalizationStrategy.RELATIONSHIP_BUILDING,
                recommended_sequence=[]
            )

        contact_name = primary_contact.get("name", "there")

        # Determine strategy
        strategy = self._determine_strategy(analysis_result)

        # Create messages based on strategy
        messages = []

        if strategy == PersonalizationStrategy.PAIN_POINT_FOCUSED:
            # Create pain point focused email
            pain_point = analysis_result.pain_points[0] if analysis_result.pain_points else None
            if pain_point:
                email_message = OutreachMessage(
                    channel=MessageChannel.EMAIL,
                    subject=f"Quick question about {pain_point.description.lower()} at {company_info.name}",
                    body=f"""Hi {contact_name},

I noticed that {company_info.name} recently {analysis_result.company_name}. This caught my attention because many {company_info.industry} companies are facing similar {pain_point.description.lower()} challenges.

I've been working with companies like yours to {pain_point.potential_solution.lower()}, and I thought you might find this interesting.

Would you be open to a brief 15-minute conversation about how other {company_info.industry} leaders are addressing {pain_point.description.lower()}?

Best regards,
{sender_name}""",
                    tone=MessageTone.CONSULTATIVE,
                    personalization_elements=[
                        f"Contact name: {contact_name}",
                        f"Company-specific pain point: {pain_point.description}",
                        f"Industry context: {company_info.industry}",
                        f"Recent company news: {analysis_result.company_name}"
                    ],
                    call_to_action="Schedule a 15-minute conversation"
                )
                messages.append(email_message)

        elif strategy == PersonalizationStrategy.OPPORTUNITY_FOCUSED:
            # Create opportunity focused email
            opportunity = analysis_result.opportunities[0] if analysis_result.opportunities else None
            if opportunity:
                email_message = OutreachMessage(
                    channel=MessageChannel.EMAIL,
                    subject=f"{company_info.name} growth opportunity - quick question",
                    body=f"""Hi {contact_name},

Congratulations on {analysis_result.company_name}! It's exciting to see {company_info.name} {opportunity.description.lower()}.

I work with {company_info.industry} companies to {opportunity.potential_value.lower()}, and I believe there might be a significant opportunity for {company_info.name} to {opportunity.category}.

Would you be interested in a brief conversation about how similar companies are {opportunity.category}?

Best regards,
{sender_name}""",
                    tone=MessageTone.PROFESSIONAL,
                    personalization_elements=[
                        f"Contact name: {contact_name}",
                        f"Company-specific opportunity: {opportunity.description}",
                        f"Industry context: {company_info.industry}",
                        f"Recent company news: {analysis_result.company_name}"
                    ],
                    call_to_action="Schedule a brief conversation"
                )
                messages.append(email_message)

        else:
            # Create relationship building email
            email_message = OutreachMessage(
                channel=MessageChannel.EMAIL,
                subject=f"Quick introduction from a fellow {company_info.industry} professional",
                body=f"""Hi {contact_name},

I hope this email finds you well. I've been following {company_info.name}'s progress in the {company_info.industry} space, particularly your work on {company_info.description}.

I'm reaching out because I work with {company_info.industry} companies to optimize their operations, and I'd love to learn more about {company_info.name}'s approach to industry challenges.

Would you be open to a brief conversation about industry trends and best practices?

Best regards,
{sender_name}""",
                tone=MessageTone.CASUAL,
                personalization_elements=[
                    f"Contact name: {contact_name}",
                    f"Company: {company_info.name}",
                    f"Industry: {company_info.industry}",
                    f"Company description: {company_info.description}"
                ],
                call_to_action="Schedule a brief conversation"
            )
            messages.append(email_message)

        # Create LinkedIn message
        linkedin_message = OutreachMessage(
            channel=MessageChannel.LINKEDIN,
            subject="Connection Request",
            body=f"Hi {contact_name}, I'd love to connect and learn more about {company_info.name}'s work in {company_info.industry}. Best regards, {sender_name}",
            tone=MessageTone.PROFESSIONAL,
            personalization_elements=[
                f"Contact name: {contact_name}",
                f"Company: {company_info.name}",
                f"Industry: {company_info.industry}"
            ],
            call_to_action="Connect and start conversation"
        )
        messages.append(linkedin_message)

        # Determine recommended sequence
        sequence = self._determine_sequence(messages, analysis_result)

        return PersonalizationResult(
            company_name=company_info.name,
            primary_contact=primary_contact,
            messages=messages,
            personalization_strategy=strategy,
            recommended_sequence=sequence
        )

    def _llm_personalization(self, company_info: CompanyInfo, analysis_result: AnalysisResult,
                           sender_name: str) -> PersonalizationResult:
        """Real LLM personalization using LangChain chains"""

        # Convert data to strings for LLM
        company_info_str = company_info.model_dump_json()
        analysis_result_str = analysis_result.model_dump_json()

        # Determine strategy using LLM
        strategy_result = self.strategy_chain.run(
            pain_points=str(analysis_result.pain_points),
            opportunities=str(analysis_result.opportunities),
            company_size=company_info.size.value
        )

        # Create messages based on strategy
        messages = []

        if "pain_point_focused" in strategy_result.lower():
            # Generate pain point focused email
            pain_point = analysis_result.pain_points[0] if analysis_result.pain_points else None
            if pain_point:
                email_result = self.email_pain_point_chain.run(
                    contact_name="there",  # Would get from contact info
                    company_name=company_info.name,
                    industry=company_info.industry,
                    pain_point=pain_point.description,
                    solution=pain_point.potential_solution,
                    sender_name=sender_name
                )
                # Parse email_result and create OutreachMessage

        # Similar logic for other strategies...

        return PersonalizationResult(
            company_name=company_info.name,
            primary_contact=None,  # Would parse from contact info
            messages=messages,
            personalization_strategy=PersonalizationStrategy.PAIN_POINT_FOCUSED,
            recommended_sequence=["linkedin_connection", "email_outreach"]
        )

    def _select_primary_contact(self, company_info: CompanyInfo, analysis_result: AnalysisResult) -> Optional[Dict[str, str]]:
        """Select the best contact for outreach"""
        if not company_info.key_contacts:
            return None

        # Simple logic: prefer CEO for startups, VP/CTO for larger companies
        if company_info.size.value == "startup":
            for contact in company_info.key_contacts:
                if "CEO" in contact.get("title", "").upper() or "Founder" in contact.get("title", "").upper():
                    return contact

        # For larger companies, prefer operational roles
        for contact in company_info.key_contacts:
            if any(role in contact.get("title", "").upper() for role in ["VP", "CTO", "COO", "DIRECTOR"]):
                return contact

        # Fallback to first contact
        return company_info.key_contacts[0]

    def _determine_strategy(self, analysis_result: AnalysisResult) -> PersonalizationStrategy:
        """Determine the best personalization strategy"""
        high_severity_pains = [p for p in analysis_result.pain_points if p.severity.value in ["high", "critical"]]
        high_priority_opps = [o for o in analysis_result.opportunities if o.priority.value in ["high", "urgent"]]

        if high_severity_pains:
            return PersonalizationStrategy.PAIN_POINT_FOCUSED
        elif high_priority_opps:
            return PersonalizationStrategy.OPPORTUNITY_FOCUSED
        else:
            return PersonalizationStrategy.RELATIONSHIP_BUILDING

    def _determine_sequence(self, messages: List[OutreachMessage], analysis_result: AnalysisResult) -> List[str]:
        """Determine recommended outreach sequence"""
        sequence = []

        # LinkedIn connection first
        linkedin_msg = next((msg for msg in messages if msg.channel == MessageChannel.LINKEDIN), None)
        if linkedin_msg:
            sequence.append("linkedin_connection")

        # Email follow-up
        email_msg = next((msg for msg in messages if msg.channel == MessageChannel.EMAIL), None)
        if email_msg:
            sequence.append("email_outreach")

        # Additional follow-up based on confidence
        if analysis_result.confidence_score > 0.7:
            sequence.append("phone_follow_up")

        return sequence

    def get_status(self) -> Dict[str, Any]:
        """Return agent status for monitoring"""
        return {
            "agent_id": self.agent_id,
            "status": "ready",
            "framework": "langchain",
            "use_mock": self.use_mock,
            "llm_available": self.llm is not None,
            "templates_configured": len(self._get_template_names()),
            "chains_configured": len(self._get_chain_names()) if not self.use_mock else 0
        }

    def _get_template_names(self) -> List[str]:
        """Get list of configured template names"""
        return ["email_pain_point_template", "email_opportunity_template", "linkedin_template", "strategy_template"]

    def _get_chain_names(self) -> List[str]:
        """Get list of configured chain names"""
        if self.use_mock:
            return []
        return ["email_pain_point_chain", "email_opportunity_chain", "linkedin_chain", "strategy_chain"]

# Example usage and testing
if __name__ == "__main__":
    print("=== LangChain Personalization Agent Demo ===\n")

    # Create agent
    personalization_agent = LangChainPersonalizationAgent(use_mock=True)

    # Test with mock data
    from langchain_models import CompanyInfo, CompanySize, AnalysisResult, PainPoint, Opportunity, PainPointSeverity, OpportunityPriority

    test_company = CompanyInfo(
        name="Acme Corporation",
        industry="Manufacturing",
        size=CompanySize.MID_MARKET,
        location="Chicago, IL",
        website="https://acmecorp.com",
        description="Leading manufacturer of industrial equipment",
        recent_news=["Expansion into European markets", "New sustainability initiative"],
        key_contacts=[{"name": "Sarah Johnson", "title": "CEO", "email": "sarah@acmecorp.com"}]
    )

    test_analysis = AnalysisResult(
        company_name="Acme Corporation",
        pain_points=[
            PainPoint(
                category="operations",
                description="Operational efficiency challenges",
                severity=PainPointSeverity.MEDIUM,
                evidence=["Mid-market company", "Growth phase"],
                potential_solution="Process optimization solutions"
            )
        ],
        opportunities=[
            Opportunity(
                category="automation",
                description="Automation and digitization opportunities",
                priority=OpportunityPriority.HIGH,
                evidence=["Manufacturing industry", "Technology adoption"],
                potential_value="Significant efficiency improvements"
            )
        ],
        industry_insights=["Manufacturing companies are increasingly focused on digital transformation"],
        recommended_approach="opportunity_focused",
        confidence_score=0.8
    )

    try:
        personalization_result = personalization_agent.personalize_outreach(
            test_company, test_analysis, "John Smith"
        )

        print(f"Personalization for: {personalization_result.company_name}")
        print(f"Strategy: {personalization_result.personalization_strategy}")
        print(f"Primary Contact: {personalization_result.primary_contact}")

        print(f"\nMessages ({len(personalization_result.messages)}):")
        for i, message in enumerate(personalization_result.messages, 1):
            print(f"\n{i}. {message.channel.upper()} Message:")
            print(f"   Subject: {message.subject}")
            print(f"   Tone: {message.tone}")
            print(f"   Body Preview: {message.body[:100]}...")
            print(f"   Personalized Elements: {', '.join(message.personalization_elements)}")

        print(f"\nRecommended Sequence:")
        for i, step in enumerate(personalization_result.recommended_sequence, 1):
            print(f"  {i}. {step}")

    except Exception as e:
        print(f"❌ Personalization failed: {str(e)}")

    print("\n" + "="*50 + "\n")

    # Show agent status
    status = personalization_agent.get_status()
    print("Agent Status:")
    for key, value in status.items():
        print(f"  {key}: {value}")




## 🔑 Key Differences from the Previous Version

### 1. **Prompt Templates**

Instead of hardcoding message strings, this version uses **LangChain `PromptTemplate`** for:

* Pain point–focused emails
* Opportunity–focused emails
* LinkedIn requests
* Strategy selection

👉 This makes the prompts **reusable, parameterized, and easy to swap** without touching the rest of the code.

---

### 2. **Chains for Modular Workflows**

Each template is wrapped in a **LangChain `LLMChain`**, so:

* You can run them independently (`email_pain_point_chain`, `linkedin_chain`)
* Or compose them into a larger workflow

👉 This modularity replaces manual branching logic with **lego-like blocks**.

---

### 3. **Structured Outputs (Pydantic Models)**

Instead of returning raw strings, outputs are wrapped in **Pydantic models** (`PersonalizationResult`, `OutreachMessage`).

* Guarantees that emails, LinkedIn messages, tone, CTAs, etc. are **structured and validated**.
* Downstream systems (CRM, Orchestrator) can consume these objects reliably.

---

### 4. **Mock Mode for Testing**

* Just like the Research and Analysis agents, this has a **`use_mock` mode**.
* Generates deterministic messages without hitting an API.
* Lets you test the **end-to-end pipeline** before plugging in real LLM calls.

---

### 5. **Strategy Selection**

The agent can automatically pick between:

* **Pain point–focused**
* **Opportunity–focused**
* **Relationship-building**

Using either:

* Mock logic (`_determine_strategy`)
* Or an LLM-driven chain (`strategy_chain`)

👉 This reduces hardcoding and adds **adaptability**.

---

### 6. **Outreach Sequencing**

It doesn’t just create messages — it also recommends a **sequence**:

1. LinkedIn connection
2. Email outreach
3. (Optional) phone follow-up if confidence is high

That’s **workflow-aware personalization**, not just message drafting.

---

## 🚀 Improvements Over the Old Version

* **Less boilerplate**: You don’t hand-roll conditionals for every scenario.
* **Modular design**: Chains and templates are swappable.
* **Safer outputs**: Structured models prevent “free-text chaos.”
* **Strategy-aware**: Outreach adapts to analysis results.
* **Production-friendly**: Easy toggling between mock mode and real LLMs.

---

## 📚 What You Should Focus on Learning

1. **Prompt engineering with `PromptTemplate`**

   * How to write prompts with placeholders (`{company_name}`, `{pain_point}`)
   * How to reuse and maintain them across chains

2. **Composability with `LLMChain`**

   * How to wire multiple chains together
   * When to run them in sequence vs. in parallel

3. **Structured outputs (Pydantic)**

   * Defining schemas for reliability
   * Parsing LLM outputs into objects

4. **Strategy logic**

   * How mock rules transition into LLM-driven reasoning
   * When to let the model decide vs. when to hardcode rules

5. **Evaluation & iteration**

   * Testing with `use_mock=True` before turning on the real LLM
   * Adding observability (logs, LangSmith traces)

---

✅ In short: This agent is less about “just sending messages” and more about **building a reliable, composable, testable system** for outreach at scale.






## 🔑 What `LLMChain` Is

* **Definition**: A LangChain **pipeline component** that binds:

  1. An **LLM** (the model you’re using)
  2. A **PromptTemplate** (structured instructions with variables)
  3. (Optionally) a parser or output key

So you don’t have to:

* Manually format prompts with `.format()`
* Call the LLM API directly
* Parse raw text responses yourself

---

## 🟦 How It Works in Your Code

Example:

```python
self.email_pain_point_chain = LLMChain(
    llm=self.llm,
    prompt=self.email_pain_point_template,
    output_key="email_message"
)
```

* `llm=self.llm` → The model (e.g., `ChatOpenAI(model="gpt-4")`)
* `prompt=self.email_pain_point_template` → A `PromptTemplate` with placeholders like `{company_name}`, `{pain_point}`
* `output_key="email_message"` → The name under which the result is stored in the returned dict

So when you run:

```python
result = self.email_pain_point_chain.run(company_name="Acme", pain_point="supply chain delays")
```

You don’t get just raw text — you get something like:

```python
{"email_message": "Hi John, I noticed Acme is facing supply chain delays..."}
```

---

## 🟦 Benefits Over Manual Code

Without `LLMChain`, you’d have to:

```python
prompt = email_pain_point_template.format(company_name="Acme", pain_point="supply chain delays")
response = llm(prompt)  # direct API call
parsed_output = {"email_message": response["text"]}
```

With `LLMChain`, that entire pattern is abstracted.

* **Less boilerplate** → one line instead of 6.
* **Consistency** → every chain behaves the same way.
* **Composability** → can drop chains into a `RunnableSequence` or pipeline without reformatting.
* **Observability** → outputs are tracked under `output_key`, making it easy to debug.

---

## 🟦 Your Setup Specifically

* `email_pain_point_chain` → generates email text focusing on pain points.
* `email_opportunity_chain` → generates email text focusing on opportunities.
* `linkedin_chain` → drafts LinkedIn connection/request message.
* `strategy_chain` → decides whether to use pain point, opportunity, or relationship-first strategy.

Each chain:

* Is a **self-contained module** (prompt + model + output key).
* Can be run independently, tested, or swapped out.
* Feeds structured output into your personalization logic.

---

✅ **Learning Focus for You**:

* How to **design PromptTemplates** for each chain (inputs & outputs).
* How `LLMChain` **abstracts prompt formatting + API call + result parsing**.
* How chains can be composed → e.g. `strategy_chain` decides which of the other chains to run.




Let’s wire up your **four chains** (`strategy_chain`, `email_pain_point_chain`, `email_opportunity_chain`, `linkedin_chain`) into a **LangChain `RunnableSequence`** so you can see the whole personalization flow at a glance.

---

## 🟦 Step 1: What You Already Have

Right now you set up four independent chains:

```python
self.email_pain_point_chain = LLMChain(llm=self.llm, prompt=self.email_pain_point_template, output_key="email_message")
self.email_opportunity_chain = LLMChain(llm=self.llm, prompt=self.email_opportunity_template, output_key="email_message")
self.linkedin_chain = LLMChain(llm=self.llm, prompt=self.linkedin_template, output_key="linkedin_message")
self.strategy_chain = LLMChain(llm=self.llm, prompt=self.strategy_template, output_key="strategy")
```

Each chain takes some input and returns a structured dict.

---

## 🟦 Step 2: Wire Them Together

```python
from langchain.schema.runnable import RunnableSequence, RunnableLambda

# Step A: Run the strategy chain first
select_strategy = self.strategy_chain

# Step B: Choose which email chain to run based on strategy
def pick_email(inputs):
    if inputs["strategy"] == "pain_point":
        return self.email_pain_point_chain.invoke(inputs)
    elif inputs["strategy"] == "opportunity":
        return self.email_opportunity_chain.invoke(inputs)
    else:
        # fallback: relationship-building strategy
        return {"email_message": f"Hi {inputs['primary_contact']}, I’d love to connect and learn more about {inputs['company_name']}."}

pick_email_chain = RunnableLambda(pick_email)

# Step C: LinkedIn chain runs in parallel
linkedin = self.linkedin_chain

# Step D: Sequence it all together
personalization_pipeline = RunnableSequence([
    select_strategy,
    pick_email_chain,
    linkedin
])
```

---

## 🟦 Step 3: Run the Pipeline

```python
result = personalization_pipeline.invoke({
    "company_name": "Acme Corp",
    "primary_contact": "Jane Doe",
    "pain_point": "supply chain delays",
    "opportunity": "automation in logistics"
})

print(result)
```

**Output (structured):**

```python
{
  "strategy": "pain_point",
  "email_message": "Hi Jane, I noticed Acme Corp has been struggling with supply chain delays...",
  "linkedin_message": "Hi Jane, I’d love to connect here on LinkedIn and share some insights..."
}
```

---

## 🔑 Why This Rocks

* **At a glance clarity**: whole personalization logic is in one pipeline.
* **No branching soup**: `RunnableLambda` makes conditional routing clean.
* **Composable**: You could drop this `personalization_pipeline` into your full Orchestrator as just one block.
* **Testable**: Run with mock data or swap out chains easily.


